In [49]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision.models as models

In [50]:
def preprocess_image(image_path, target_size=(256, 256)):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Convert grayscale image to 3 channels
    img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    
    img_rgb = cv2.resize(img_rgb, target_size)
    img_rgb = img_rgb / 255.0
    img_pil = Image.fromarray((img_rgb * 255).astype(np.uint8))
    return img_pil

In [51]:
def get_name_and_class():
    directory_path = os.getcwd()
    files = os.listdir(directory_path + '/dataset')
    image_files = sorted([file for file in files if file.lower().endswith(('.jpeg'))], key=lambda x: int(x.split('.')[0]))
    full_paths = [os.path.join(directory_path + '/dataset/', file) for file in image_files]

    classes = []
    classes.extend(["chavoshi"] * 20)
    classes.extend(["shajarian"] * 23)
    classes.extend(["khaliq"] * 21)
    classes.extend(["radan"] * 25)
    classes.extend(["bayati"] * 21)
    classes.extend(["kianafshar"] * 25)
    classes.extend(["alidoosti"] * 27)
    classes.extend(["qaforian"] * 25)
    classes.extend(["razavian"] * 20)
    classes.extend(["daei"] * 27)
    classes.extend(["attaran"] * 45)
    classes.extend(["beiranvand"] * 32)
    classes.extend(["dolatshahi"] * 24)
    classes.extend(["esfahani"] * 25)
    classes.extend(["hoceini"] * 20)
 
    return full_paths,classes

In [52]:
image_paths,labels = get_name_and_class()
X_train, X_temp, y_train, y_temp = train_test_split(image_paths, labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [53]:
class_to_label = {
    "chavoshi": 0,
    "shajarian": 1,
    "khaliq": 2,
    "radan": 3,
    "bayati": 4,
    "kianafshar": 5,
    "alidoosti": 6,
    "qaforian": 7,
    "razavian": 8,
    "daei": 9,
    "attaran": 10,
    "beiranvand": 11,
    "dolatshahi": 12,
    "esfahani": 13,
    "hoceini": 14,
}

In [54]:
class FaceDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = [class_to_label[label] for label in labels]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = preprocess_image(self.image_paths[idx])
        label = self.labels[idx]
        label = torch.tensor(label, dtype=torch.long) 
        if self.transform:
            image = self.transform(image)
        return image, label

In [55]:
transform = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
])
train_dataset = FaceDataset(X_train, y_train, transform=transform)
val_dataset = FaceDataset(X_val, y_val, transform=transform)
test_dataset = FaceDataset(X_test, y_test, transform=transform)

In [56]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [57]:
class FaceRecognitionModel(nn.Module):
    def __init__(self, num_classes):
        super(FaceRecognitionModel, self).__init__()

        # Load pre-trained ResNet-18 model
        resnet18 = models.resnet18(pretrained=True)
        # Remove the last fully connected layer
        self.resnet_layers = nn.Sequential(*list(resnet18.children())[:-1])

        # Add your custom layers with increased capacity
        self.fc1 = nn.Linear(512, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
        self.batch_norm1 = nn.BatchNorm1d(1024)
        self.batch_norm2 = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Pass input through ResNet layers
        x = self.resnet_layers(x)
        x = F.leaky_relu(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.relu(self.batch_norm1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.batch_norm2(self.fc2(x)))
        x = self.fc3(x)
        return x

In [58]:
model = FaceRecognitionModel(num_classes=len(set(labels)))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=8, gamma=0.1)

c:\Users\Roham\Desktop\ml-project\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Roham\Desktop\ml-project\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [59]:
epochs = 10
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        labels = torch.tensor(labels, dtype=torch.long)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Learning rate scheduler step
    scheduler.step(loss)  # Pass loss to scheduler for dynamic adjustment

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            labels = torch.tensor(labels, dtype=torch.long)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_accuracy = correct / total
        average_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch + 1}/{epochs} | Validation Loss: {average_val_loss:.4f} | Validation Accuracy: {val_accuracy * 100:.2f}%")


C:\Users\Roham\AppData\Local\Temp\ipykernel_15424\2654636550.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)
c:\Users\Roham\Desktop\ml-project\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
C:\Users\Roham\AppData\Local\Temp\ipykernel_15424\2654636550.py:23: UserWarning: To copy construct from a tensor, it is 

Epoch 1/12 | Validation Loss: 2.5018 | Validation Accuracy: 21.05%
Epoch 2/12 | Validation Loss: 2.1931 | Validation Accuracy: 25.00%
Epoch 3/12 | Validation Loss: 1.9370 | Validation Accuracy: 34.21%
Epoch 4/12 | Validation Loss: 1.7704 | Validation Accuracy: 44.74%
Epoch 5/12 | Validation Loss: 1.8624 | Validation Accuracy: 52.63%
Epoch 6/12 | Validation Loss: 2.0851 | Validation Accuracy: 38.16%
Epoch 7/12 | Validation Loss: 2.0633 | Validation Accuracy: 40.79%
Epoch 8/12 | Validation Loss: 1.4420 | Validation Accuracy: 60.53%
Epoch 9/12 | Validation Loss: 1.4149 | Validation Accuracy: 60.53%
Epoch 10/12 | Validation Loss: 1.6593 | Validation Accuracy: 53.95%
Epoch 11/12 | Validation Loss: 1.6303 | Validation Accuracy: 51.32%
Epoch 12/12 | Validation Loss: 1.7780 | Validation Accuracy: 43.42%


In [60]:
torch.save(model.state_dict(), "fine_face_recognition_model.pth")

In [ ]:
model.load_state_dict(torch.load("fine_face_recognition_model.pth"))
evaled = model.eval()

In [62]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 50.00%


Usage

In [68]:
# Load the model
model = FaceRecognitionModel(num_classes=15)
model.load_state_dict(torch.load("fine_face_recognition_model.pth"))
model.eval()

# Define the transform for preprocessing the input image
image_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

def predict_class(image_path):
    image = Image.open(image_path)
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)
    return predicted_class.item()

# Example usage
image_paths_to_predict = ["test.jpeg", ]
exam_labels = [1,]
correct = 0
for i,image in enumerate(image_paths_to_predict):
    predicted_class = predict_class(image)
    if predict_class==exam_labels[i]:
        correct += 1
    # # Map the predicted class index to the corresponding label
    # index_to_class = {v: k for k, v in class_to_label.items()}
    
    # predicted_label = index_to_class[predicted_class]
    # print(f"Predicted Class: {predicted_label}",)
accuracy = correct / len(image_paths_to_predict)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

c:\Users\Roham\Desktop\ml-project\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Roham\Desktop\ml-project\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Test Accuracy: 0.00%
